# Optimized Model Profiling

This notebook queries the option data, runs the optimized Black-76 model (vectorized), and profiles its execution performance.

In [1]:
import sys
import os
import pandas as pd
import cProfile
import pstats
import io
import time
from pstats import SortKey

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from query.csv_loader import load_all_csv_data, filter_by_time_range
from optimization.optimized_model import calculate_optimized

## 1. Load Data

In [2]:
try:
    df = load_all_csv_data(data_dir="../data")
    print("Successfully loaded.")
    filter_df = filter_by_time_range(df=df, start_time="09:30", end_time="10:00")
except Exception as e:
    print(f"Error loading data: {e}")
    raise e

Successfully loaded.


## 2. Profile Optimized Model

We use `cProfile` to profile the `calculate_optimized_metrics` function.

In [3]:
print("Starting optimized execution with profiling...")

# Initialize profiler
pr = cProfile.Profile()
pr.enable()

start_bench = time.time()

# Run the optimized model
result_df = calculate_optimized(filter_df)

end_bench = time.time()
pr.disable()

total_time = end_bench - start_bench
print(f"Optimization completed in {total_time:.4f} seconds.")

Starting optimized execution with profiling...
Optimization completed in 0.0297 seconds.


## 2.1. Benchmark Multiple Runs

Run `calculate_optimized` multiple times and calculate average execution time.


In [9]:
# Run calculate_optimized multiple times and get average execution time
num_runs = 20
execution_times = []

print(f"Running calculate_optimized {num_runs} times...")
for i in range(num_runs):
    start_time = time.time()
    result_df = calculate_optimized(filter_df)
    end_time = time.time()
    execution_time = end_time - start_time
    execution_times.append(execution_time)
    print(f"Run {i + 1}/{num_runs}: {execution_time:.4f} seconds")

# Calculate statistics
avg_time = sum(execution_times) / len(execution_times)
min_time = min(execution_times)
max_time = max(execution_times)
std_time = pd.Series(execution_times).std()

print(f"\n--- Benchmark Results ---")
print(f"Number of runs: {num_runs}")
print(f"Average execution time: {avg_time:.4f} seconds")
print(f"Minimum execution time: {min_time:.4f} seconds")
print(f"Maximum execution time: {max_time:.4f} seconds")
print(f"Standard deviation: {std_time:.4f} seconds")

Running calculate_optimized 20 times...
Run 1/20: 0.0165 seconds
Run 2/20: 0.0148 seconds
Run 3/20: 0.0144 seconds
Run 4/20: 0.0139 seconds
Run 5/20: 0.0136 seconds
Run 6/20: 0.0148 seconds
Run 7/20: 0.0141 seconds
Run 8/20: 0.0139 seconds
Run 9/20: 0.0131 seconds
Run 10/20: 0.0134 seconds
Run 11/20: 0.0137 seconds
Run 12/20: 0.0131 seconds
Run 13/20: 0.0127 seconds
Run 14/20: 0.0116 seconds
Run 15/20: 0.0126 seconds
Run 16/20: 0.0141 seconds
Run 17/20: 0.0136 seconds
Run 18/20: 0.0133 seconds
Run 19/20: 0.0125 seconds
Run 20/20: 0.0116 seconds

--- Benchmark Results ---
Number of runs: 20
Average execution time: 0.0136 seconds
Minimum execution time: 0.0116 seconds
Maximum execution time: 0.0165 seconds
Standard deviation: 0.0011 seconds


## 3. Profiling Results

In [5]:
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats(20)
print(s.getvalue())

         11766 function calls (11290 primitive calls) in 0.029 seconds

   Ordered by: cumulative time
   List reduced from 613 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.011    0.005 /Users/jg/Documents/mlp_projects/optimization/black76_model.py:302(calculate_implied_volatility)
        2    0.006    0.003    0.011    0.005 /Users/jg/Documents/mlp_projects/optimization/black76_model.py:207(_calculate_iv_newton)
        2    0.000    0.000    0.008    0.004 /Users/jg/Documents/mlp_projects/venv/lib/python3.14/site-packages/pandas/core/tools/datetimes.py:678(to_datetime)
        2    0.000    0.000    0.005    0.003 /Users/jg/Documents/mlp_projects/venv/lib/python3.14/site-packages/pandas/core/tools/datetimes.py:211(_maybe_cache)
        2    0.000    0.000    0.003    0.002 /Users/jg/Documents/mlp_projects/venv/lib/python3.14/site-packages/pandas/core/tools/datetimes.py:316(_convert_listlike_da

## 4. Data Preview

In [6]:
result_df[
    [
        "Date",
        "ExpirationDate",
        "Strike",
        "CallPut",
        "VWAP",
        "ImpliedVol",
        "Delta",
        "Gamma",
        "Vega",
        "Theta",
        "Rho",
    ]
].head()

,Date,ExpirationDate,Strike,CallPut,VWAP,ImpliedVol,Delta,Gamma,Vega,Theta,Rho
0,20200129,20200131,1540.0,P,0.1300,0.958028,-0.003177,0.000073,1.328770,-116.245736,-0.000712
1,20200129,20200131,1787.5,P,8.0667,0.604821,-0.175869,0.003103,35.620441,-1967.549638,-0.044171
2,20200129,20200131,1535.0,P,0.1300,0.972831,-0.003132,0.000071,1.311683,-116.523925,-0.000712
3,20200129,20200131,1760.0,P,3.9457,0.606228,-0.098221,0.002072,23.878474,-1321.975093,-0.021605
4,20200129,20200131,1730.0,P,1.9500,0.625942,-0.052695,0.001246,14.821594,-847.223426,-0.010678
